In [2]:
!pip install pandas scikit-learn
!pip install kaggle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 152.2 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.6.14-py3-none-any.whl size=105119 sha256=7e1bb6df43102a2a471ac39b09308ac67733e63dd68d9f53a6094ece9712f347
  Stored in directory: /home/asu/.cache/pip/wheels/d7/54/06/8a8f40cb39536605feb9acaacd0237a95eba39e5065e6392f4
Successfully built kaggle


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# 加载数据
train_data = pd.read_csv('./titanic/train.csv')
test_data = pd.read_csv('./titanic/test.csv')


In [5]:
print(train_data.head())
print(train_data.info())
print(train_data.describe())


   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  
<c

In [6]:
# 处理缺失值
train_data['Age'].fillna(train_data['Age'].median(), inplace=True)
train_data['Embarked'].fillna(train_data['Embarked'].mode()[0], inplace=True)
train_data.drop(['Cabin'], axis=1, inplace=True)  # Cabin列缺失值过多，直接删除

# 对测试集进行同样的处理
test_data['Age'].fillna(test_data['Age'].median(), inplace=True)
test_data['Fare'].fillna(test_data['Fare'].median(), inplace=True)
test_data.drop(['Cabin'], axis=1, inplace=True)


In [7]:
# 将性别和登船港口转为数值
train_data = pd.get_dummies(train_data, columns=['Sex', 'Embarked'], drop_first=True)
test_data = pd.get_dummies(test_data, columns=['Sex', 'Embarked'], drop_first=True)

# 选择特征
features = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Sex_male', 'Embarked_Q', 'Embarked_S']
X = train_data[features]
y = train_data['Survived']
X_test = test_data[features]


In [8]:
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_test = scaler.transform(X_test)


In [10]:
print(X)
print(X.shape)

[[ 0.82737724 -0.56573646  0.43279337 ...  0.73769513 -0.30756234
   0.61583843]
 [-1.56610693  0.66386103  0.43279337 ... -1.35557354 -0.30756234
  -1.62380254]
 [ 0.82737724 -0.25833709 -0.4745452  ... -1.35557354 -0.30756234
   0.61583843]
 ...
 [ 0.82737724 -0.1046374   0.43279337 ... -1.35557354 -0.30756234
   0.61583843]
 [-1.56610693 -0.25833709 -0.4745452  ...  0.73769513 -0.30756234
  -1.62380254]
 [ 0.82737724  0.20276197 -0.4745452  ...  0.73769513  3.25137334
  -1.62380254]]
(891, 8)


In [12]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [13]:
model = LogisticRegression()
model.fit(X_train, y_train)


LogisticRegression()

In [14]:
y_pred = model.predict(X_val)
print(f"Accuracy: {accuracy_score(y_val, y_pred)}")
print(classification_report(y_val, y_pred))


Accuracy: 0.8100558659217877
              precision    recall  f1-score   support

           0       0.83      0.86      0.84       105
           1       0.79      0.74      0.76        74

    accuracy                           0.81       179
   macro avg       0.81      0.80      0.80       179
weighted avg       0.81      0.81      0.81       179



In [15]:
test_predictions = model.predict(X_test)


In [16]:
# 生成提交文件
submission = pd.DataFrame({
    "PassengerId": test_data["PassengerId"],
    "Survived": test_predictions
})

In [17]:
submission.to_csv('submission.csv', index=False)


In [19]:
!pip install autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 kB 1.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 kB 2.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.4/109.4 kB 2.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 4.5 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 1.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of requests[socks] to determine wh

In [20]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset


In [37]:
class MLP(nn.Module):
    def __init__(self, input_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 32)
        self.fc3 = nn.Linear(32, 32)
        self.fc4 = nn.Linear(32, 32)
        self.fc5 = nn.Linear(32, 32)
        self.fc6 = nn.Linear(32, 32)
        self.fc7 = nn.Linear(32, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.relu(self.fc4(x))
        x = self.relu(self.fc5(x))
        x = self.relu(self.fc6(x))
        x = self.sigmoid(self.fc7(x))
        return x

input_size = X_train.shape[1]
model = MLP(input_size)


In [38]:
# 转换为 PyTorch 张量
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32).view(-1, 1)

# 创建数据加载器
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [39]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(train_loader.dataset)
    return epoch_loss

def evaluate(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            preds = outputs.round()
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    epoch_loss = running_loss / len(val_loader.dataset)
    accuracy = correct / total
    return epoch_loss, accuracy

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

num_epochs = 300
best_accuracy = 0.0

for epoch in range(num_epochs):
    train_loss = train(model, train_loader, criterion, optimizer, device)
    val_loss, val_accuracy = evaluate(model, val_loader, criterion, device)
    
    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        torch.save(model.state_dict(), 'best_model.pth')
    
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

print(f'Best Validation Accuracy: {best_accuracy:.4f}')


Epoch 1/300, Train Loss: 0.6830, Val Loss: 0.6767, Val Accuracy: 0.5866
Epoch 2/300, Train Loss: 0.6474, Val Loss: 0.5966, Val Accuracy: 0.7654
Epoch 3/300, Train Loss: 0.5168, Val Loss: 0.4725, Val Accuracy: 0.7709
Epoch 4/300, Train Loss: 0.4760, Val Loss: 0.4382, Val Accuracy: 0.8101
Epoch 5/300, Train Loss: 0.4361, Val Loss: 0.4198, Val Accuracy: 0.8156
Epoch 6/300, Train Loss: 0.4265, Val Loss: 0.4156, Val Accuracy: 0.8156
Epoch 7/300, Train Loss: 0.4134, Val Loss: 0.4317, Val Accuracy: 0.7989
Epoch 8/300, Train Loss: 0.4072, Val Loss: 0.4229, Val Accuracy: 0.8156
Epoch 9/300, Train Loss: 0.3965, Val Loss: 0.4206, Val Accuracy: 0.8380
Epoch 10/300, Train Loss: 0.3952, Val Loss: 0.4277, Val Accuracy: 0.8268
Epoch 11/300, Train Loss: 0.3941, Val Loss: 0.4196, Val Accuracy: 0.8324
Epoch 12/300, Train Loss: 0.3905, Val Loss: 0.4260, Val Accuracy: 0.8324
Epoch 13/300, Train Loss: 0.3893, Val Loss: 0.4280, Val Accuracy: 0.8212
Epoch 14/300, Train Loss: 0.3874, Val Loss: 0.4231, Val Accu

In [40]:
# 加载最佳模型
model.load_state_dict(torch.load('best_model.pth'))

# 转换测试集为 PyTorch 张量
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

# 使用模型进行预测
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor.to(device))
    test_preds = test_outputs.round().cpu().numpy()

# 生成提交文件
submission = pd.DataFrame({
    "PassengerId": test_data["PassengerId"],
    "Survived": test_preds.flatten().astype(int)
})
submission.to_csv('submission2.csv', index=False)

print("Submission file created successfully.")


Submission file created successfully.
